## Bibliotecas

In [20]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import numpy as np
import math
import operator
from sklearn.feature_extraction.text import TfidfVectorizer

## Consulta do Usuário

In [21]:
def ConsultaUsuário(consulta):
    
    My_stopwords = set(stopwords.words('english'))
    
    # Separar a consulta por palavras.
    palavras = word_tokenize(consulta)
    
    # Remover as Stopwords.
    lista_Termos = [i for i in palavras if i not in My_stopwords]
    lista_Termos = list(dict.fromkeys(lista_Termos))
    
    # Stemming para pegar termos similares atraves dos radicais das palavras.
    ps = PorterStemmer()
    lista_radicais = []
    for w in lista_Termos:
        if ps.stem(w) not in lista_radicais: 
            lista_radicais.append(ps.stem(w))

    return lista_Termos

# Exemplos de Teste
* Modelo do Indice Invertido [ Termo , [ ( DOCUMENTO , TF/IDF ) ] ]
* Modelo da Consulta do Usuário: Texto qualquer

In [22]:
Docs = [
    ['Black',[('doc1',1),('doc2',2)]],
    ['dragon',[('doc2',3),('doc3',1)]],
    ['Illusion',[('doc4',1)]]
]

consulta = r'Black Lotus'

## Modelo Espaço Vetorial

### Similaridade Cosseno de Vetores

In [23]:
# Similaridade Cosseno
def SimilaridadeCosseno(vetorQuery,vetorDoc):
    score = np.dot(vetorQuery,vetorDoc) / (math.sqrt((np.dot(vetorQuery,vetorQuery))*(np.dot(vetorDoc,vetorDoc))))
    return score

### Matrix de Vetores
* <strong>Sem TF/IDF</strong> 
    Apenas a presença do termo no documento.

In [24]:
def MatrixVetores(Postings,query):
    
    # Lista com os termos tanto dos postings quanto da consulta.
    termosVocabulario = []
    
    # Add consulta(query) e os documentos.
    documentoNomes = [query]
    
    # Add termos da consulta ao vocabulario.
    for termos in ConsultaUsuário(query):
        if termos not in termosVocabulario:
            termosVocabulario.append(termos)
    
    # Add os termos dos documentos dos postings ao vocabulario.
    for posting in Postings:
        if posting[0] not in termosVocabulario:
            termosVocabulario.append(posting[0])
        for tupla in posting[1]:
            if tupla[0] not in documentoNomes:
                documentoNomes.append(tupla[0])
    
    #print('termos do Vocabulario (Query e Documentos): ',termosVocabulario,"\n\n")
    #print('Nome dos Documentos (Query e Documentos): ',documentoNomes,"\n\n")
    
    # Criando matrix de vetores 
    matrixVazia = np.empty((len(termosVocabulario),len(documentoNomes)),dtype=object)
    
    # Add nomes as rows e columns (rows = termos e columns docs)
    df = pd.DataFrame(matrixVazia, columns=documentoNomes, index=termosVocabulario)
    
    # Inserir os valores na matrix (sem tf/idf) se existe 1 se não 0
    nameColumns = list(df.columns)
    nameRows = list(df.index)
     
    # Preenchendo os valores dos vetores usando Document-at-time.
    for doc in nameColumns:
        for termo in nameRows:
            
            # Verificando a query.
            if termo in ConsultaUsuário(query) and doc == query:
                df.loc[termo,doc] = 1
            else:
                df.loc[termo,doc] = 0
     
            # Verificando os docs.
            for posting in Postings:
                if termo == posting[0]:
                    for tupla in posting[1]:
                        if doc == tupla[0]:
                            if tupla[1] > 0:
                                df.loc[termo,doc] = 1
                            else:
                                df.loc[termo,doc] = 0
      
    
    df.rename(columns={query:'Query'},inplace = True)    
    #display(df)
    return df
MatrixVetores(Docs,consulta)

,Query,doc1,doc2,doc3,doc4
Black,1,1,1,0,0
Lotus,1,0,0,0,0
dragon,0,0,1,1,0
Illusion,0,0,0,0,1


### Matrix de Vetores
* <strong>TF/IDF</strong> 

In [31]:
def MatrixVetores_TF_IDF(Postings,query):
    
    # Lista com os termos tanto dos postings quanto da consulta.
    termosVocabulario = []
    
    # Add consulta(query) e os documentos.
    documentoNomes = [query]
    
    # Add termos da consulta ao vocabulario.
    for termos in ConsultaUsuário(query):
        if termos not in termosVocabulario:
            termosVocabulario.append(termos)
    
    # Add os termos dos documentos dos postings ao vocabulario.
    for posting in Postings:
        if posting[0] not in termosVocabulario:
            termosVocabulario.append(posting[0])
        for tupla in posting[1]:
            if tupla[0] not in documentoNomes:
                documentoNomes.append(tupla[0])
    
    #print('termos do Vocabulario (Query e Documentos): ',termosVocabulario,"\n\n")
    #print('Nome dos Documentos (Query e Documentos): ',documentoNomes,"\n\n")
    
    # Criando matrix de vetores 
    matrixVazia = np.empty((len(termosVocabulario),len(documentoNomes)),dtype=object)
    
    # Add nomes as rows e columns (rows = termos e columns docs)
    df = pd.DataFrame(matrixVazia, columns=documentoNomes, index=termosVocabulario)
    
    # Inserir os valores na matrix (sem tf/idf) se existe 1 se não 0
    nameColumns = list(df.columns)
    nameRows = list(df.index)
     
    # Preenchendo os valores dos vetores usando Document-at-time.
    for doc in nameColumns:
        for termo in nameRows:
            
            # Verificando a query.
            if termo in ConsultaUsuário(query) and doc == query:
                #TF/IDF
               #inicializarVetorTfIdf = TfidfVectorizer()
               #TF_IDF = inicializarVetorTfIdf.fit_transform([query])
                
                # por enquanto só TF
                TF = ConsultaUsuário(query).count(termo)/len(ConsultaUsuário(query))
                df.loc[termo,doc] = TF
            else:
                df.loc[termo,doc] = 0
     
            # Verificando os docs.
            for posting in Postings:
                if termo == posting[0]:
                    for tupla in posting[1]:
                        if doc == tupla[0]:
                            if tupla[1] > 0:
                                # valor do 2 componente da tupla [termo , [ ( doc, TF/IDF ) ] ]
                                df.loc[termo,doc] = tupla[1]
                            else:
                                df.loc[termo,doc] = 0
      
    
    df.rename(columns={query:'Query'},inplace = True)    
    #display(df)
    return df
MatrixVetores_TF_IDF(Docs,consulta)

,Query,doc1,doc2,doc3,doc4
Black,0.5,1,2,0,0
Lotus,0.5,0,0,0,0
dragon,0,0,3,1,0
Illusion,0,0,0,0,1


### Respota Consulta 
    *Lista de Documentos Ordenada pela similaridade dos cossenos (Documento e Query)

In [32]:
# Resposta
def RespostaConsulta(matrixVetores):
    
    # Lista de cocumentos e seus scores (doc,score)
    ListaDocumentos = []
    
    # Pegando o vetorQuery
    vetorQuery = matrixVetores['Query'].to_numpy()
    
    for col in matrixVetores.columns:  
        # Score da Similaridade de cossenos
        score = SimilaridadeCosseno(vetorQuery,matrixVetores[col].to_numpy())
        
        # documento comparado a query
        documento = col

        ListaDocumentos.append((documento,score))
    
    # Ordenar pelo Score.
    ListaDocumentos.sort(key=operator.itemgetter(1),reverse=True)
    
    return ListaDocumentos

## Resposta usando MatrixVetores sem TF/IDF

In [33]:
RespostaConsulta(MatrixVetores(Docs,consulta))

[('Query', 1.0),
 ('doc1', 0.7071067811865475),
 ('doc2', 0.5),
 ('doc3', 0.0),
 ('doc4', 0.0)]

## Resposta usando MatrixVetores com TF/IDF

In [34]:
RespostaConsulta(MatrixVetores_TF_IDF(Docs,consulta))

[('Query', 1.0),
 ('doc1', 0.7071067811865475),
 ('doc2', 0.3922322702763681),
 ('doc3', 0.0),
 ('doc4', 0.0)]

In [70]:
# Correlação de Spearman
def Correlação_Spearman():
    return